In [3]:
# TESTING Z/7Z

for t in range(-5, 5):
    # Copy and paste a4, a6, r, and lower_bound here:
    a4 = -27*t^16 + 216*t^15 + 216*t^14 - 4536*t^13 + 5292*t^12 + 10584*t^11 - 58968*t^10 - 735480*t^9 + 275454*t^8 + 4246344*t^7 - 12070296*t^6 + 24698520*t^5 - 42681492*t^4 + 49601160*t^3 - 37633896*t^2 + 35429400*t - 22851963
    a6 = 54*t^24 - 648*t^23 + 648*t^22 + 17928*t^21 - 58644*t^20 - 84888*t^19 + 646056*t^18 - 4663656*t^17 + 18256266*t^16 + 63974448*t^15 - 301043952*t^14 + 402730704*t^13 + 609484392*t^12 - 2170409904*t^11 + 5384487312*t^10 - 17208004176*t^9 + 38179206954*t^8 - 82948098600*t^7 + 175384505448*t^6 - 284139536472*t^5 + 378103265388*t^4 - 421713313848*t^3 + 353901442248*t^2 - 206968634568*t + 64311801174
    r = 1
    lower_bound = 2^(-r) * 7.0/sqrt((18.0*log(t)+7.0))
    
    print("hi")
    try:
        E = EllipticCurve([a4, a6])
    except:
        print("there was some error (ie. singular curve or two descent function failed) for t=", t)
        continue
    
    print("a4 =", a4)
    print("a6 =", a6)
    if get_c(E, 1) <= lower_bound:
        print("c(E) lower bound is bad for t =", t)
    else:
        print("c(E) lower bound is good for t =", t)

hi
a4 = -2744325439488
a6 = 4859195654053822464
c(E) lower bound is good for t = -5
hi
a4 = 48738455973
a6 = 13088850280632054


KeyboardInterrupt: 

In [1]:
#Compute constants c(E), c_hat(E) from Griffin-Ono-Tsai
def get_c(E, r):
    '''
    Input
        E: elliptic object
        r: E's rank
    Output
        c(E) from new G-O-T paper
    '''
    if r is None:
        r = E.rank()
    
    omega_r = pi^(r/2) / gamma(r/2 + 1)
    E_tor = E.torsion_order()
    R_Q = E.regulator()
    
    c = E_tor* omega_r / (2^(r+1) * R_Q^(1.0/2))
    return c

#TESTING
E_coeffs = [0, 0, 0, 0, -174]
E = EllipticCurve(E_coeffs)
print(N(get_c(E, 3)))

0.0385559424470825


In [2]:
ParametrizedPolynomial.<t> = PolynomialRing(ZZ)
NumericalConstant = RealField(32)
LogPolynomial.<u> = PolynomialRing(NumericalConstant)
ParametrizedRationalFunction = ParametrizedPolynomial.fraction_field()
ParametrizedEllipticCurvePolynomial.<x, y> = PolynomialRing(ParametrizedRationalFunction)

def is_first_larger(f, g):
    if f == 0 or g == 0:
        print("One of the functions 'f' and 'g' is zero!")
        return
    if f.degree() != g.degree():
        return f.degree() > g.degree()
    if f.leading_coefficient() < 0:
        f = -f
    if g.leading_coefficient() < 0:
        g = -g
    return f.leading_coefficient() >= g.leading_coefficient()

def size_compare_range(f, g):
    if f == 0 or g == 0:
        print("One of the functions 'f' and 'g' is zero!")
        return
    if f.leading_coefficient() < 0:
        f = -f
    if g.leading_coefficient() < 0:
        g = -g
    k = 0
    while any([c < 0 for c in f.substitute(t=t+k).coefficients()]) or any([c < 0 for c in g.substitute(t=t+k).coefficients()]) or any([c < 0 for c in (f-g).substitute(t=t+k).coefficients()]):
        k += 1
    if (-1)^f.degree()*f.leading_coefficient() < 0:
        f = -f
    if (-1)^g.degree()*g.leading_coefficient() < 0:
        g = -g
    j = 0
    while any([c < 0 for c in f.substitute(t=-t-j).coefficients()]) or any([c < 0 for c in g.substitute(t=-t-j).coefficients()]) or any([c < 0 for c in (f-g).substitute(t=-t-j).coefficients()]):
        j += 1
    while k > -j and f.substitute(t=k-1) >= 0 and g.substitute(t=k-1) >= 0 and (f-g).substitute(t=k-1) >= 0:
        k -= 1
    while j > -k and f.substitute(t=-j+1) >= 0 and g.substitute(t=-j+1) >= 0 and (f-g).substitute(t=-j+1) >= 0:
        j -= 1
    return (-j, k)

def get_height_approx(f, c):
    f = ParametrizedRationalFunction(f)
    num = f.numerator(); denom = f.denominator()
    num_larger = is_first_larger(num, denom)
    if num_larger:
        monomial = abs(num.leading_coefficient())*t^num.degree()
    else:
        monomial = abs(denom.leading_coefficient())*t^denom.degree()
    monomial *= c
    lower1, upper1 = size_compare_range(monomial, num)
    lower2, upper2 = size_compare_range(monomial, denom)
    return (monomial, min(lower1, lower2), max(upper1, upper2))

def compute_constant_lower_bound(elliptic_curve_polynomial, elliptic_curve_points, torsion_size, constant_factor):
    a4, a6 = WeierstrassForm(elliptic_curve_polynomial, variables = [x,y])
    a4 *= 6^4; a6 *= 6^6; E = EllipticCurve([a4, a6])
    print("a4 = " + str(a4))
    print()
    print("a6 = " + str(a6))
    print()
    A, B, C = WeierstrassForm(elliptic_curve_polynomial, variables = [x,y], transformation=True)
    X = 6^2*A/C^2; Y = 6^3*B/C^3
    r = len(elliptic_curve_points)
    xs = [X.substitute(x=P[0], y=P[1]) for P in elliptic_curve_points]
    for i in range(r):
        print("xs[" + str(i) + "] = " + str(xs[i]))
        print()
    lower_cutoff = +infinity; upper_cutoff = -infinity
    xhs = []
    for i in range(r):
        monomial, lower, upper = get_height_approx(xs[i], constant_factor)
        lower_cutoff = min(lower_cutoff, lower); upper_cutoff = max(upper_cutoff, upper)
        xhs += [monomial]
    for i in range(r):
        print("xhs[" + str(i) + "] = " + str(xhs[i]))
        print()
        
    j_invariant = ParametrizedRationalFunction(E.j_invariant())
    print("j_invariant = " + str(j_invariant))
    print()
    jh, lower, upper = get_height_approx(j_invariant, constant_factor)
    lower_cutoff = min(lower_cutoff, lower); upper_cutoff = max(upper_cutoff, upper)
    print("jh = " + str(jh))
    print()
    
    discriminant = ParametrizedRationalFunction(E.discriminant())
    print("discriminant = " + str(discriminant))
    print()
    dh, lower, upper = get_height_approx(discriminant, constant_factor)
    lower_cutoff = min(lower_cutoff, lower); upper_cutoff = max(upper_cutoff, upper)
    print("dh = " + str(dh))
    print()
    
    print("(lower_cutoff, upper_cutoff) = " + str((lower_cutoff, upper_cutoff)))
    print()
    
    regulator_polynomial = 1
    for i in range(r):
        regulator_polynomial *= ((1/12)*log(jh.leading_coefficient()) + (1/12)*log(dh.leading_coefficient()) + (1/2)*log(xhs[i].leading_coefficient()) + log(3)) + ((1/12)*jh.degree() + (1/12)*dh.degree() + (1/2)*xhs[i].degree())*u
    print("regulator_polynomial = " + str(LogPolynomial(regulator_polynomial)))
    print()
    
    lower_bound_const = torsion_size*floor(pi^(r/2)/gamma(r/2+1))*2^(-r)
    
    print("lower bound for c(E): " + str(NumericalConstant(lower_bound_const)) + "/sqrt(" + str(LogPolynomial(regulator_polynomial)) + ")")
    return